## Model path

In [6]:
LLM_MODEL_PATH = "D:\\Generative AI\\Gen AI Language\\local LLM\\llama2-7b-q2chat.bin"
EMBEDDING_MODEL_PATH ="D:\\Generative AI\\Gen AI Language\\local LLM\\local embedding\\sentence-transformers-local\\all-MiniLM-L6-V2"
SQLITE_DATABASE_PATH=""

## Loading LLM Locally

In [7]:
from langchain.llms import CTransformers
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = CTransformers(
    model = LLM_MODEL_PATH,
    model_type = "llama",
    max_new_tokens = 200,
    temperature = 0.7,
)

template = """Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt = prompt, llm = llm)

In [8]:
llm_chain.invoke({
    "question": "Who are you?"
})

{'question': 'Who are you?',
 'text': "I am an AI assistant developed by Meta AI. I'm just an AI, my purpose is to assist and provide helpful responses to your questions. I can answer a wide range of topics, from simple queries to more complex subjects. Feel free to ask me anything!"}

## Loading Embedding Model Locally
First download the model and save it locally.

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name = EMBEDDING_MODEL_PATH
)

In [10]:
embedding.embed_query("Hello how are you?")[:10]

[0.0028318532276898623,
 0.03901991993188858,
 0.08902649581432343,
 0.07300351560115814,
 -0.026130713522434235,
 -0.0733506828546524,
 0.055322688072919846,
 -0.010837383568286896,
 -0.08899175375699997,
 0.017466789111495018]

## SQL Chain

In [12]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///products.db")

### 1: SQL Agent

In [17]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(
    llm, 
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True,
)

### 2: SQL Chain

In [21]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# sql query chain: generates sql query based on given prompt
sql_query_chain = create_sql_query_chain(llm, db)

# sql validator chain: validate the generated query from sql query chain
system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

The database contains: 
table: 'products'
with columns in 'products' table:
'name': <name of the product>
'image_url':<url for the product image>
'price': <price of the product>
'category': <category of the product (mobile, tv, camera)>
'description': <more information about the product>
'stock': <current stock items in numbers>

If there are any of the above mistakes, rewrite the query.
If there are no mistakes, just reproduce the original query with no further commentary.
If the query is Data Modification type then return "Data modification is not allowed" as the response.

otherwise:
Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

sql_chain = {"query": sql_query_chain} | validation_chain


In [ ]:
# testing the chain
sql_chain.invoke({
    "question": "How many products are there?"
})

In [23]:
# answer chain
from langchain_core.output_parsers import StrOutputParser

answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding  SQL query, and the result of the query, answer the question in a short and concise manner.
    If asked about product price, description response accordingly only by looking at the <result>. If asked about stock inquiry response according to the sql result. (if 0 then stock is currently unavailable so user cannot order.)
    
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer:
    """
)

answer_chain = answer_prompt | llm | StrOutputParser()

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

query_executor_chain = QuerySQLDataBaseTool(db=db)

final_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(result=itemgetter("query") | query_executor_chain)
    | answer_chain
)